[Link to tutorial page](https://juliaai.github.io/DataScienceTutorials.jl/getting-started/ensembles/)

# Ensemble models

## Preliminary steps

Generate dummy data.

In [5]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Repos/mike_scratch/mlj_tutorial/A-ensembles`


┌ Warning: The active manifest file has dependencies that were resolved with a different julia version (1.7.1). Unexpected behavior may occur.
└ @ nothing /Users/mph/Repos/mike_scratch/mlj_tutorial/A-ensembles/Manifest.toml:0


   Installed PooledArrays ────────── v1.4.0


   Installed NearestNeighbors ────── v0.4.9
   Installed StaticArrays ────────── v1.3.2


   Installed NearestNeighborModels ─ v0.1.6
   Installed DataFrames ──────────── v1.3.1


...

  ✓ StaticArrays


  ✓ NearestNeighbors


  ✓ NearestNeighborModels


  ✓ DataFrames


  5 dependencies successfully precompiled in 10 seconds. 91 already precompiled.
  2 dependencies errored. To see a full report either run `import Pkg; Pkg.precompile()` or load the packages


In [6]:
using MLJ
import DataFrames: DataFrame
using PrettyPrinting
using StableRNGs

In [7]:
rng = StableRNG(512)
Xraw = rand(rng, 300, 3)
y = exp.(Xraw[:,1] - Xraw[:,2] - 2Xraw[:,3] + 0.1*rand(rng,300))
X = DataFrame(Xraw, :auto)

train, test = partition(eachindex(y), 0.7)

([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  201, 202, 203, 204, 205, 206, 207, 208, 209, 210], [211, 212, 213, 214, 215, 216, 217, 218, 219, 220  …  291, 292, 293, 294, 295, 296, 297, 298, 299, 300])

In [8]:
# load a simple model
KNNRegressor = @load KNNRegressor
knn_model = KNNRegressor(K=10)

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/mph/.julia/packages/MLJModels/tMgLW/src/loading.jl:168


import NearestNeighborModels ✔

KNNRegressor(
    K = 10,
    algorithm = :kdtree,
    metric = Distances.Euclidean(0.0),
    leafsize = 10,
    reorder = true,
    weights = NearestNeighborModels.Uniform())

In [9]:
knn = machine(knn_model, X, y)

Machine{KNNRegressor,…} trained 0 times; caches data
  model: NearestNeighborModels.KNNRegressor
  args: 
    1:	Source @255 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @220 ⏎ `AbstractVector{Continuous}`


In [10]:
fit!(knn, rows=train)
ŷ = predict(knn, rows=test)
rms(ŷ, y[test])

┌ Info: Training Machine{KNNRegressor,…}.
└ @ MLJBase /Users/mph/.julia/packages/MLJBase/MuLnJ/src/machines.jl:464


0.06389980172436367

In [11]:
# We could have done the following instead.
evaluate!(knn, resampling=Holdout(fraction_train=0.7, rng=StableRNG(666)), measure=rms)

PerformanceEvaluation object with these fields:
  measure, measurement, operation, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_pairs
Extract:
┌────────────────────────┬─────────────┬───────────┬──────────┐
│ measure                │ measurement │ operation │ per_fold │
├────────────────────────┼─────────────┼───────────┼──────────┤
│ RootMeanSquaredError() │ 0.124       │ predict   │ [0.124]  │
└────────────────────────┴─────────────┴───────────┴──────────┘


## Homogenous ensebles